In [1]:
import pickle
from requests import patch
from tqdm import tqdm
from PIL import Image
import numpy as np
import os
import random 
import pickle
import re

In [2]:
def select_frames_by_patches(video_path,frame_list,num):
    """ Randomly select frames according to the number of image patches that a video needs

    Args:
        video_path ( str ): the path of video frame
        frame_list ( str ): the list of video frame
        num ( int ): the number of patches that a video needs
    """
    
    img  = Image.open(os.path.join(video_path,frame_list[0]))
    n = (img.size[0]//256)*(img.size[1]//256)
    n_frames = int(num/n)
    if len(frame_list) < n_frames:
        return frame_list
    return random.sample(frame_list,n_frames)

In [3]:
def split_image(img):
    
    """ Spilit the PIL Image to patches"""

    cropped = []
    size = img.size
    w = size[0]//256
    h =  size[1]//256
    for i in range(w):
        for j in range(h):
            box = (256 * i, 256 * j, 256 * (i + 1), 256 * (j + 1))
            region = img.crop(box)
            cropped.append(np.array(region))
    return cropped

In [4]:
def convert_img_2_pickle(img,save_path):
    """ convert image patches into pickles

    Args:
        img (_type_): image patch
        path (_type_): the save path of patch
    """
    with open(save_path,'wb') as f:
        pickle.dump(img,f)

In [5]:
def find_consecutive_P_frames(i_frames_no, P_list):
    """ find the consecutive frames after certain I frame

    Args:
        i_frames_no (int): idx of I frame
        P_list (list): the list of P frames

    Returns:
        _type_: The first P frame after the I frame
    """
    i_frames_no =int(i_frames_no.split('.')[0]) 
    for i in range(len(P_list)):
        if P_list[i] > i_frames_no:
            return i

In [6]:
def rename_patch(img_name,ftype):
    l = img_name.split('.jpg')[0].split('/')
    return '{}_{}_{}_{}'.format(l[0],l[1],ftype,l[2])

In [7]:
def create_pairs(frame_dir,save_text_path,num_of_P_frames=3,num_of_patches=140):
    """ create I frame and P frame patches and out the pair info to specific text file

    Args:
        frame_dir (str): the path of frames dir
        save_text_path (str): the save path of output text file
        num_of_P_frames (int, optional): the number of consecutive P frames. Defaults to 3.
        num_of_patches (int, optional): the number of patches a video needs. Defaults to 140.
    """
    # 首先获取所有视频列表
    video_list = os.listdir(frame_dir)
    # 再对每个视频单独处理
    # 首先选取视频的I帧
    with open(os.path.join(save_text_path),"a+") as f:
        for video in video_list:
            I_frame_path = os.path.join(frame_dir,video,'I')
            # 读取I帧的列表
            I_frame_list = os.listdir(I_frame_path)
            # 根据需要切出的图像块的个数来选择I帧
            selected_I_frames = select_frames_by_patches(I_frame_path,I_frame_list,num_of_patches)
            # 根据Iframe来选择Pframes
            # 首先获取P帧列表
            P_frame_path = os.path.join(frame_dir,video,'P')
            P_frame_list = os.listdir(P_frame_path)\
            # 对每个I帧，选择其随后的n个P帧
            for i in range(len(P_frame_list)):
                P_frame_list[i] = int(P_frame_list[i].split('.jpg')[0])
            P_frame_list.sort()
            len_p_frames = len(P_frame_list)
            for f_no in selected_I_frames:
                p_idx = find_consecutive_P_frames(f_no,P_frame_list) 
                if p_idx ==None:
                    print(os.path.join(video,'I',str(f_no)+'.jpg'))
                # 如果I帧后存在n个P帧，则将其写入txt中
                else:
                    if (p_idx+num_of_P_frames) < len_p_frames:
                        f.writelines(os.path.join(video,'I',str(f_no))+'\n')
                        for i in range(num_of_P_frames):
                            f.writelines(os.path.join(video,'P',str(P_frame_list[p_idx+i]))+'.jpg\n')
            

In [8]:
def create_dataset(txt_path,frame_dir,label,save_dir):
    """ crete dataset

    Args:
        txt_path (_type_): the path of text file that contains pairs info about I frames and P frames
        frame_dir (_type_): the path of frames dir
        label (_type_): the label of frames
        save_dir (_type_): the save path of created dataset
    """
    # 首先读取保存I帧P帧对的文本
    frame_list = [] 
    pair = []
    with open(txt_path,"r") as f: 
        count = 1
        for item in f.readlines():
            pair.append(item.strip())
            if count==3:
                frame_list.append(pair)
                pair = []
                count = 1
            else:
                count+=1
    # 接下来对进行切片处理
    video_name = ''
    for frames in tqdm(frame_list):
        if frames[0].split('/')[0] != video_name:
            video_name = frames[0].split('/')[0]
            n = 0
        I_patch = split_image(Image.open(os.path.join(frame_dir,frames[0])))
        P_patch = []
        for i in range(1,4):    
            P_patch.append(split_image(Image.open(os.path.join(frame_dir,frames[0]))))
        I_name = rename_patch(frames[0],label)
        P_name_1 = rename_patch(frames[1],label)
        P_name_2 = rename_patch(frames[2],label)
        P_name_3 = rename_patch(frames[3],label)
        for j in range(len(I_patch)):
            name_1 = '{}_{}.pickle'.format(I_name,j)
            name_2 = '{}_{}.pickle'.format(P_name_1,j)
            name_3 = '{}_{}.pickle'.format(P_name_2,j)
            name_4 = '{}_{}.pickle'.format(P_name_3,j)
            convert_img_2_pickle(I_patch[j],os.path.join(save_dir,name_1))
            convert_img_2_pickle(P_patch[0][j],os.path.join(save_dir,name_2))
            convert_img_2_pickle(P_patch[1][j],os.path.join(save_dir,name_3))
            convert_img_2_pickle(P_patch[2][j],os.path.join(save_dir,name_4))
            with open(os.path.join(save_dir,'FB_test.txt'),'a') as f:
                f.writelines('{} {} {} {} {} {}\n'.format(name_1,name_2,name_3,name_4,label))
    

In [14]:
create_pairs("./data/frames/FB/","FB.txt")

In [9]:
create_dataset('FB.txt','./data/frames/FB','FB','/data/frames/patches/train')

100%|██████████| 716/716 [01:05<00:00, 10.94it/s]
